# 1st way - pure bs4, 1 link per match, very unefficient

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

In [6]:
def scrap_data(url):
    soup = BeautifulSoup(requests.get(url, headers={"User-Agent":"Mozilla/5.0"}).content, 'html.parser')
    teams = soup.findAll('a', attrs = {'class' : 'team-title'})
    scores = soup.findAll('h3', attrs = {'class' : 'thick scoretime'})
    if len(scores)>0:
        title=soup.find('title').text.split()
        FT=re.findall(r'\d+', str(scores))[1:3]
        HT=re.findall(r'\d+', str(scores))[3:5]
        teamA=teams[0].text
        teamB=teams[1].text
        #date=title[-5:-2]
        date=soup.find('span', attrs = {'class' : 'timestamp'}).text
        #league=soup.find('li', attrs = {'class' : 'expanded current odd'}).find('a').text
        league=soup.find('div', attrs = {'class' : 'details'}).findAll('a')[1].text
        return FT,HT,teamA,teamB,league,date

In [7]:
links=["https://us.soccerway.com/matches/2001/09/07/asia/wc-qualifying-asia/iraq/iran/"+str(i) for i in range(1,4000000)]

# 2nd way - selenium+bs4, much faster, but might miss some (very few) matches

In [2]:
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
#driver = webdriver.Chrome(ChromeDriverManager().install())
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

            #for headless
from selenium.webdriver.chrome.options import Options
options = Options()
#options.add_argument('--headless')
options.add_argument('--user-agent="Mozilla/5.0 (Windows Phone 10.0; Android 4.2.1; Microsoft; Lumia 640 XL LTE) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Mobile Safari/537.36 Edge/12.10166"')
#options.add_argument('--disable-gpu')  # Last I checked this was necessary.
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
from selenium.common.exceptions import ElementClickInterceptedException

[WDM] - Downloading: 100%|████████████████| 6.58M/6.58M [00:00<00:00, 8.22MB/s]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


In [4]:
def scrap_matches(link): #date should be given in format /yyyy/mm/dd/
    
    inRow=0
    driver.get(link)
    #time.sleep(4)
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'livescores')))
    
    for i in range(1,1000):
        button=driver.find_elements_by_css_selector(f'#livescores > div.livescores-comps > div:nth-child({i}) > h2 > button')
        try:
            button[0].click()
            #driver.execute_script("arguments[0].click();", button[0])
            inRow=0
        except IndexError:
            inRow+=1
            if inRow==2:
                break
        except ElementClickInterceptedException:
            continue
    
    print("converting to bs4")
    print("...")
    time.sleep(5)
    soup_results_overview = BeautifulSoup(driver.page_source, 'html')
    matches=soup_results_overview.findAll('div', attrs = {'class' : 'matchinfo'})
    try:
        infos=[match.find('a', href=True)['href'].split("/") for match in matches]
        dates=[info[2:5] for info in infos]
        competitions=[info[5:7] for info in infos]
        homes=[info[7] for info in infos]
        aways=[info[8] for info in infos]
        FTs_all=[match.findAll('span', attrs = {'class' : 'team_score'}) for match in matches]
        homeFTs=[FTs[0].text for FTs in FTs_all]
        awayFTs=[FTs[1].text for FTs in FTs_all]
        HTs_all=[match.findAll('span', attrs = {'class' : 'team_htscore'}) for match in matches]
        homeHTs=[HTs[0].text for HTs in HTs_all]
        awayHTs=[HTs[1].text for HTs in HTs_all]
        results=list(zip(homes,aways,homeFTs,awayFTs,homeHTs,awayHTs,dates,competitions))
        print(f"results length {len(results)}")
        return results
    except IndexError:
        pass

In [ ]:
"""

def scrap_matches(link): #date should be given in format /yyyy/mm/dd/
    
    st = time.time()
    inRow=0
    driver.get(link)
    #time.sleep(4)
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'livescores')))
    
    for i in range(1,1000):
        button=driver.find_elements_by_css_selector(f'#livescores > div.livescores-comps > div:nth-child({i}) > h2 > button')
        try:
            button[0].click()
            #driver.execute_script("arguments[0].click();", button[0])
            inRow=0
        except IndexError:
            inRow+=1
            if inRow==2:
                break
        except ElementClickInterceptedException:
            continue
    
    print("converting to bs4")
    print("...")
    time.sleep(5)
    soup_results_overview = BeautifulSoup(driver.page_source, 'html')
    matches=soup_results_overview.findAll('div', attrs = {'class' : 'matchinfo'})
    
    infos=[match.find('a', href=True)['href'].split("/") for match in matches]
    dates=[info[2:5] for info in infos]
    competitions=[info[5:7] for info in infos]
    homes=[info[7] for info in infos]
    aways=[info[8] for info in infos]
    FTs_all=[match.findAll('span', attrs = {'class' : 'team_score'}) for match in matches]
    homeFTs=[FTs[0].text for FTs in FTs_all]
    awayFTs=[FTs[1].text for FTs in FTs_all]
    HTs_all=[match.findAll('span', attrs = {'class' : 'team_htscore'}) for match in matches]
    homeHTs=[HTs[0].text for HTs in HTs_all]
    awayHTs=[HTs[1].text for HTs in HTs_all]
    results=list(zip(homes,aways,homeFTs,awayFTs,homeHTs,awayHTs,dates,competitions))
    print(f"results length {len(results)}")
    et = time.time()
    elapsed_time = et - st
    print('Execution time:', elapsed_time, 'seconds')
    
    return results
    
    """

In [5]:
driver.get("https://us.soccerway.com/matches/2001/10/06/") #USE IT TO  MANUALLY ACCEPT COOKIES

In [3]:
days=['0'+str(i) if i<10 else str(i) for i in range(1,32)]
jan=["https://us.soccerway.com/matches/2016/01/"+day for day in days]
feb=["https://us.soccerway.com/matches/2016/02/"+day for day in days[:-2]]
mar=["https://us.soccerway.com/matches/2016/03/"+day for day in days]
apr=["https://us.soccerway.com/matches/2016/04/"+day for day in days[:-1]]
may=["https://us.soccerway.com/matches/2016/05/"+day for day in days]
jun=["https://us.soccerway.com/matches/2016/06/"+day for day in days[:-1]]
jul=["https://us.soccerway.com/matches/2016/07/"+day for day in days]
aug=["https://us.soccerway.com/matches/2016/08/"+day for day in days]
sep=["https://us.soccerway.com/matches/2016/09/"+day for day in days[:-1]]
otb=["https://us.soccerway.com/matches/2016/10/"+day for day in days]
nov=["https://us.soccerway.com/matches/2016/11/"+day for day in days[:-1]]
dec=["https://us.soccerway.com/matches/2016/12/"+day for day in days]
y2016=jan+feb+mar+apr+may+jun+jul+aug+sep+otb+nov+dec

In [8]:
results=[]

In [ ]:
for day in y2016[135:]:
    results.append(scrap_matches(day))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  # Remove the CWD from sys.path while we load stuff.


converting to bs4
...
results length 60
converting to bs4
...
results length 332
converting to bs4
...
results length 175
converting to bs4
...
results length 349
converting to bs4
...
results length 212
converting to bs4
...
results length 337
converting to bs4
...
results length 1458
converting to bs4
...
results length 1162
converting to bs4
...
results length 170
converting to bs4
...
results length 168
converting to bs4
...
results length 377
converting to bs4
...
results length 185
converting to bs4
...
results length 252
converting to bs4
...
results length 1182
converting to bs4
...
results length 920
converting to bs4
...
results length 130
converting to bs4
...
results length 103
converting to bs4
...
results length 188
converting to bs4
...
results length 144
converting to bs4
...
results length 135
converting to bs4
...
results length 903
converting to bs4
...
results length 641
converting to bs4
...
results length 118
converting to bs4
...
results length 102
converting to 

In [57]:
len(results)

366

In [59]:
flat=[]
for day in results:
    try:
        for match in day:
            flat.append(match)
    except TypeError:
        pass

In [38]:
flat

[('sheffield-united-fc',
  'manchester-city-football-club',
  '0',
  '1',
  '0',
  '1',
  ['2020', '10', '31'],
  ['england', 'premier-league']),
 ('burnley-fc',
  'chelsea-football-club',
  '0',
  '3',
  '0',
  '1',
  ['2020', '10', '31'],
  ['england', 'premier-league']),
 ('liverpool-fc',
  'west-ham-united-fc',
  '2',
  '1',
  '1',
  '1',
  ['2020', '10', '31'],
  ['england', 'premier-league']),
 ('football-club-dallas',
  'houston-dynamo',
  '3',
  '0',
  '2',
  '0',
  ['2020', '10', '31'],
  ['united-states', 'mls']),
 ('bristol-city-fc',
  'norwich-city-fc',
  '1',
  '3',
  '1',
  '3',
  ['2020', '10', '31'],
  ['england', 'championship']),
 ('afc-bournemouth',
  'derby-county-fc',
  '1',
  '1',
  '0',
  '1',
  ['2020', '10', '31'],
  ['england', 'championship']),
 ('barnsley-fc',
  'watford-football-club',
  '1',
  '0',
  '1',
  '0',
  ['2020', '10', '31'],
  ['england', 'championship']),
 ('luton-town-fc',
  'brentford-fc',
  '0',
  '3',
  '0',
  '2',
  ['2020', '10', '31'],
 

In [60]:
len(flat)

133536

In [61]:
df = pd.DataFrame(np.array(flat), columns = ['home','away','ft_home','ft_away','ht_home','ht_away','date','league'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [62]:
df.head()

,home,away,ft_home,ft_away,ht_home,ht_away,date,league
0,brighton--hove-albion-fc,chelsea-football-club,1,1,0,1,"[2020, 01, 01]","[england, premier-league]"
1,burnley-fc,aston-villa-football-club,1,2,0,2,"[2020, 01, 01]","[england, premier-league]"
2,newcastle-united-football-club,leicester-city-fc,0,3,0,2,"[2020, 01, 01]","[england, premier-league]"
3,southampton-fc,tottenham-hotspur-football-club,1,0,1,0,"[2020, 01, 01]","[england, premier-league]"
4,watford-football-club,wolverhampton-wanderers-fc,2,1,1,0,"[2020, 01, 01]","[england, premier-league]"


In [63]:
df.to_csv('2016.csv', index=False) 